# TODO
0. add levenshtein distance to naive model to get best predictions
0. during the submit process, save all new dataset names in the naive model.
0. after NLP prediction, use levenshtein distance to find previously known dataset. If distance is small enough, use the known dataset
1. do data pre-processing (cleaning etc) IMPORTANT: cleaning the whole text may result in inconsistencies between the dataset name in our cleaned text, and the dataset label in the answers csv
3. add naive baseline (with a buffer of all known datasets) and combine with spacy model
4. add SciBERT as a third model


In [3]:
!nvcc --version
!pip install spacy

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:15:10_Pacific_Standard_Time_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107102 sha256=b797b800390af18fdc2dfd6087fde5cd5484284e56ffe7774ede6388f999380c
  Stored in directory: c:\users\31640\appdata\local\pip\cache\wheels\83\a6\12\bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attempting uninstall: click
    Found existing installation: Click 7.0
    Uninstalling Click-7.0:
      Successfully uninstalled Click-7.0


In [2]:
import spacy
spacy.require_gpu() # !IMPORTANT: turn this off when gpu not turned on

ModuleNotFoundError: No module named 'spacy'

# imports

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import csv
import pickle
from math import floor
from random import shuffle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(f"{dirname} contains {len(filenames)} files")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input contains 0 files
/kaggle/input/coleridgeinitiative-show-us-the-data contains 2 files
/kaggle/input/coleridgeinitiative-show-us-the-data/test contains 4 files
/kaggle/input/coleridgeinitiative-show-us-the-data/train contains 14316 files
/kaggle/input/mlip2-spacy contains 6 files


# GLOBAL SETTINGS

In [5]:
saved_model_filepath = "../input/mlip2-spacy/2021-05-04 21:11:55.931579.sav" # SET TO 'None' if you want to train from scratch!
save_model = True

# utilities

In [6]:
import string 
import re

def clean_text(text):
    '''
    Converts all text to lower case, Removes special characters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    text = re.sub(' +', ' ', text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

# load data


In [7]:
def load_train_data():
    training_data = []
    
    # open the csv with id's, data labels, etc. and append the json files to it
    files = []
    train_dir = '../input/coleridgeinitiative-show-us-the-data/train' # location of the training json files
    df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv') # location of the training csv file (does not contain the actual texts)
    for i in df.index:
        file_id = df['Id'][i]
        filename = f"{file_id}.json"
        filepath = os.path.join(train_dir, filename)
        with open(filepath) as json_file:
            file = json.loads(json_file.read())
            files.append(file)
    df['file'] = files
    
    return df

# df = load_train_data()
# df.describe()

In [5]:
# some config variables
MIN_LENGTH_SAMPLE = 10
MAX_LENGTH_SAMPLE = 9000

def format_dataframe_for_spacy(df):
    data = []
    for i in df.index:
        # dataset name, to be used as training label
        file = df['file'][i]
        dataset_title = df['dataset_title'][i]

        for section in file:
            # each section contains a 'section_title' and a 'text' key, for now we only use 'text'
            text = section['text']

            # only consider this sample when these rules apply
            if len(text) < MIN_LENGTH_SAMPLE:
                continue
            if len(text) > MAX_LENGTH_SAMPLE:
                continue
            # END OF RULES

            # !IMPORTANT TODO: Adding padding to the dataset title removes about 1/3rd of the training data. probably not good
            if f' {dataset_title} ' in text: # Only use a section as a training sample IF it contains a dataset label
                start_index = text.find(dataset_title)
                end_index = start_index + len(dataset_title)
                entity = (start_index, end_index, 'DATASET')
                entities = [entity]
                sample = (text, {'entities': entities})
                data.append(sample)
                # TODO: What if it finds two matches in a file?
                        
    return data

spacy_data = format_dataframe_for_spacy(df)
print(spacy_data[0])

('This study used data from the National Education Longitudinal Study (NELS:88) to examine the effects of dual enrollment programs for high school students on college degree attainment. The study also reported whether the impacts of dual enrollment programs were different for first generation college students versus students whose parents had attended at least some college. In addition, a supplemental analysis reports on the impact of different amounts of dual enrollment course-taking and college degree attainment.\nDual enrollment programs offer college-level learning experiences for high school students. The programs offer college courses and/or the opportunity to earn college credits for students while still in high school.\nThe intervention group in the study was comprised of NELS participants who attended a postsecondary school and who participated in a dual enrollment program while in high school (n = 880). The study author used propensity score matching methods to create a compa

In [8]:
def load_test_data():
    test_data = {}
    for dirname, _, filenames in os.walk('../input/coleridgeinitiative-show-us-the-data/test'):
        for filename in filenames:
            filepath = os.path.join(dirname, filename)
            with open(filepath) as json_file:
                file = json.loads(json_file.read())
                file_id = filename.replace(".json", "")
                test_data[file_id] = file
    return test_data

TEST_DATA = load_test_data()
print(TEST_DATA.keys())

dict_keys(['8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60', '2100032a-7c33-4bff-97ef-690822c43466', '2f392438-e215-4169-bebf-21ac4ff253e1', '3f316b38-1a24-45a9-8d8c-4e05a42257c6'])


In [7]:
# train_len = floor(len(spacy_data) * 0.9)
# TRAIN_DATA = spacy_data[:train_len]
# VAL_DATA = spacy_data[train_len:]
# print(len(TRAIN_DATA))
# print(len(VAL_DATA))

# TRAIN_DATA = spacy_data

# Spacy NER model

In [9]:
def create_blank_nlp(train_data):
    nlp = spacy.load("en_core_web_sm")
#     nlp = spacy.blank("en") # TODO: Either create a blank one and add a pipe, or download one and replace the NER pipe...
    nlp.remove_pipe("ner") # If creating from blank, this line of code should be commented
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

In [40]:
import datetime as dt

def load_nlp_from_file(filename):
    with open(filename, 'rb') as pickle_file:
        model = pickle.load(pickle_file)
        return model
    
def save_nlp_to_file(model):
    filename = f"{dt.datetime.now()}.sav"
    pickle.dump(model, open(filename, 'wb'))
    return filename

In [45]:
# !pip show spacy
spacy.load

<function spacy.load(name, **overrides)>

In [54]:
# save_nlp_to_file(nlp)
nlp = load_nlp_from_file("./bytes_data.sav")
text = "The spur of ICT (Information and Communication Technologies) innovations in the twenty-first century has massively disrupted economies and business models (Christensen, 2013; Tohmatsu, 2012) . Millions of jobs face a high probability of being replaced because computers and the internet are reshaping the labor market (Oliver, 2015) . In the framework of globalization, digital skills are now considered preliminary for securing professional employment across the globe (Pirzada & Khan, 2013) . Consequently, many employers across a wide range of sectors are increasingly viewing ICT skills as an important component of employability (Belt & Richardson, 2005; Johnson & Burden, 2003) . The omnipresence of computing has made digital literacy increasingly critical to success in any occupation (Murray & P\u00e9rez, 2014) . In fact, organizations are now identifying digital skills or computer literacy as one of their core values for employability (such as the US Department of Education, the US Department of commerce, the OECD Program for the International Assessment of Adult Competencies and the European Commission).\nIn particular, in developing countries, digital skills are estimated to reduce poverty levels and increase employment rates (Bennett, Maton, & Kervin, 2008) . ICT competence can bring significant benefits to marginalized groups, allowing these groups to participate more fully in society as it improves employment opportunities, overcomes isolation, builds confidence and leads to further learning (Bunker, 2010) . It\"s not a surprise that Digital Literacy has been deemed an essential life skill (Europea, 2008) . Individuals\" digital engagement has direct implications on their academic performance, labor market success and entrepreneurship uptakes. Those who function better in the digital realm and participate more fully in digitally mediated life enjoy advantages over their digitally disadvantaged counterparts (Robinson et al., 2015) . ICT have become central to every economy and to people\"s quality of life in every society (Olatoye, 2011) .\nNevertheless, a new form of inequality is emerging, adding to all the existing forms of discrimination (Hilbert, 2011) . Concerns are being raised that the digital divide is leaving behind those most in need of assistance (Greig, 2004) . It is feared that as the role of ICT continues to expand, the exclusion experienced by disadvantaged groups may be accentuated and reinforced, rather than mitigated (Corrigan & Joyce, 2000) . Discussions about digital divides typically refer to socio-economic disparities in accessibility and usage of ICT. The use of such technologies result in several beneficial outcomes while the non-use of such technologies excludes people from full participation in contemporary society (Helsper, Deursen, & Eynon, 2015) . Many studies have established that basic access to digital resources and the skills to effectively use them still escape many economically disadvantaged or traditionally underrepresented portions of the population (Witte & Mannon, 2010) . Thus, digital inequalities can reinforce and exacerbate existing social inequalities (DiMaggio & Garip, 2012) . Further interventions are needed to ensure that the unqualified, the low skilled, the long-term unemployed and those on low incomes are enabled to reap the benefits of new services and opportunities for job seeking through ICT (Lindsay, 2005) ."
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f9cc0dbc7d0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f9cc0a450c0>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f9cc09dd7c0>)]

In [30]:
from spacy.util import minibatch, compounding
from tqdm import trange

EPOCHS = 3

def train_nlp_model():
    nlp = create_blank_nlp(TRAIN_DATA)
    optimizer = nlp.begin_training()
    t = trange(EPOCHS, desc='After first epoch, this will show losses', leave=True)
    for i in t:
        losses = {}
        batch_gen = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        batches = list(batch_gen)
        batch_counter = 0
        for batch in batches:
            t.set_description(f'epoch: {i} | batch {batch_counter} of {len(batches)}')
            batch_counter += 1
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.1,  # dropout - make it harder to memorise data
                losses=losses,
            )
        t.write(f"Losses at iteration {i} - {dt.datetime.now()} {losses}")
    return nlp

    
nlp = None
if saved_model_filepath:
    print(f"Loading model from: {saved_model_filepath}...")
    nlp = load_nlp_from_file(saved_model_filepath)
    print("Loaded model")
else:
    print("Training model from scratch...")
    nlp = train_nlp_model()
    print("Model trained")
    if save_model:
        print("Saving trained model...")
        save_nlp_to_file(nlp)
        print("Model saved")

Loading model from: ../input/mlip2-spacy/2021-05-04 21:11:55.931579.sav...
Loaded model


# Naive model
keeps track of all dataset labels in our training data, and checks if a sample contains one of these, and if so, returns it

In [ ]:
class NaiveModel():
    def __init__(self, df):
        self.labels = []
        self.df = df
        
    def train(self):
        for i in self.df.index:
            label = self.df["dataset_title"][i]
            if not label in self.labels:
                self.labels.append(label)
            
        
    def predict(self, file):
        for section in file:
            for label in self.labels:
                  if label.lower() in section['text'].lower():
                    return label

Create naive model and train it

In [ ]:
naive = NaiveModel(df)
naive.train()
print(f'Naive model has collected: {len(naive.labels)} unique labels')

# Generate output file

utility class to get insight into some meta data

In [ ]:
class Counter():
    def __init__(self):
        self.reset()
    
    def increment_naive(self):
        self.naive += 1
        
    def increment_nlp(self):
        self.nlp += 1
        
    def reset(self):
        self.naive = 0
        self.nlp = 0

predict function that takes in some models, and tries to predict a dataset title for each

In [ ]:
def predict_one(naive, nlp, file, counter):
    # naive model first
    label = naive.predict(file)
    if label:
        counter.increment_naive()
        return label
    
    return "NOT FOUND" # !IMPORTANT TODO: REMOVE THIS
    
    # if naive model cannot find it, use spacy model
    for section in file:
        doc = nlp(section['text'])
        for ent in doc.ents:
            # TODO: this ONLY CONSIDERS THE FIRST ONE. This first guess might be wrong, and the actual filename might be found further in the doc
            if ent.label_ == "DATASET":
                counter.increment_nlp()
                return ent.text
            
    # If none of the models find anything
    return "NOT FOUND"

In [ ]:
counter = Counter()

# temp code for checking stuff / debugging
# temp_test_set = {}
# for i in df.index:
#     if (len(temp_test_set.items()) > 100):
#         continue
#     key = df["Id"][i]
#     file = df["file"][i]
#     temp_test_set[key] = file

def submit(data):
    buffer = {}
    for file_id, file in data.items():
        dataset_label = predict_one(naive, nlp, file, counter)
        buffer[file_id] = dataset_label
    
    ids = buffer.keys()
    predictions = buffer.values()
    data = {"Id": ids, "PredictionString": predictions }
    df = pd.DataFrame(data=data)
    df.to_csv('submission.csv', index=False)
    
    # print some meta data
    print(f'naive predicted: {counter.naive}')
    print(f'nlp predicted: {counter.nlp}')

# submit(temp_test_set)
submit(TEST_DATA)